# 🧪 Notebook 01: Datengenerierung für AOI-THT-Inspektion
Dieses Notebook erzeugt einen simulierten Datensatz zur Lötstelleninspektion mittels AOI-System.
Es werden synthetische Merkmale wie Volumen, Höhe, Rundheit usw. erzeugt – aufgeteilt in OK- und NOK-Bauteile mit einem Verhältnis von 4:1 (80 % OK, 20 % NOK).

In [1]:
import pandas as pd
import numpy as np

In [3]:
# Parameter
np.random.seed(97)
n_samples = 500
ok_ratio = 0.8
n_ok = int(n_samples * ok_ratio)
n_nok = n_samples - n_ok

In [5]:

# OK-Daten simulieren
  #loc: Mittelwert,
  #scale: Standardabweichung 
  #size: Anzahl der Werte

ok_data = {
    'convex_volume': np.random.normal(loc=2.5, scale=0.3, size=n_ok),
    'max_height': np.random.normal(loc=1.5, scale=0.2, size=n_ok),
    'area': np.random.normal(loc=4.0, scale=0.4, size=n_ok),
    'roundness': np.random.normal(loc=0.9, scale=0.05, size=n_ok),
    'pad_distance': np.random.normal(loc=0.2, scale=0.05, size=n_ok),
    'label': ['OK'] * n_ok
}

# NOK-Daten simulieren
nok_data = {
    'convex_volume': np.random.normal(loc=4.5, scale=0.5, size=n_nok),
    'max_height': np.random.normal(loc=2.5, scale=0.3, size=n_nok),
    'area': np.random.normal(loc=5.5, scale=0.6, size=n_nok),
    'roundness': np.random.normal(loc=0.6, scale=0.1, size=n_nok),
    'pad_distance': np.random.normal(loc=0.05, scale=0.02, size=n_nok),
    'label': ['NOK'] * n_nok
}


In [12]:
# Daten kombinieren & mischen
df_ok = pd.DataFrame(ok_data)
df_nok = pd.DataFrame(nok_data)
df = pd.concat([df_ok, df_nok]).sample(frac=1, random_state=97).reset_index(drop=True)
df = df.round(4)
df.head(10)

,convex_volume,max_height,area,roundness,pad_distance,label
0,1.9264,1.4259,3.8846,0.8988,0.2280,OK
1,2.6191,1.5801,4.2819,0.9030,0.1525,OK
2,2.9427,1.2019,3.3805,0.8941,0.2546,OK
3,2.6678,1.7501,4.1928,0.8484,0.1524,OK
4,4.0634,2.0017,6.3782,0.5019,0.0802,NOK
5,2.3926,1.2957,3.7398,0.8317,0.1974,OK
6,2.2502,1.4045,3.6152,0.9566,0.2624,OK
7,2.6682,1.4022,3.5797,0.8346,0.1802,OK
8,3.0173,1.1952,3.8982,0.8690,0.2108,OK
9,2.4635,1.3904,3.4636,0.8393,0.1685,OK


In [7]:
df.label.value_counts(normalize=True)

label
OK     0.8
NOK    0.2
Name: proportion, dtype: float64

In [9]:
output_file = "01_generated_aoi_data_80_20_ratio_rounded.csv"
df.to_csv(output_file, index=False)
print(f"Datei gespeichert unter: {output_file}")

Datei gespeichert unter: 01_generated_aoi_data_80_20_ratio_rounded.csv



## 📊 Merkmalsbeschreibung des AOI-Datensatzes

Im Folgenden werden die 5 wichtigsten numerischen Merkmale beschrieben, die zur Klassifikation von THT-Lötfehlern verwendet werden:

| Spalte           | Bedeutung | Praxisbezug                                                                                             | Typischer Unterschied: OK vs. NOK |
|------------------|-----------|---------------------------------------------------------------------------------------------------------|------------------------------------|
| `convex_volume`  | **Volumen der Lotstruktur** (in mm³) | Misst, wie viel Lötzinn vorhanden ist. Hohe Volumenwerte deuten oft auf Lotkugeln hin.                  | ✅ OK: ~2–3 mm³<br>❌ NOK: ~4–6 mm³ |
| `max_height`     | **Maximale Höhe** des Objekts (Z-Achse) | Zeigt die Erhebung – Lotkugeln ragen meist höher hervor.                                       | ✅ OK: ~1.5 mm<br>❌ NOK: ~2.5 mm |
| `area`           | **Fläche der Projektion** (in mm²) | Wie groß erscheint das Objekt von oben? Glanz oder Flussmittel können dies beeinflussen.                | ✅ OK: kompakt<br>❌ NOK: großflächig |
| `roundness`      | **Formfaktor** (0–1) | Gibt an, wie rund das Objekt ist. Lotkugeln sind kugelförmig, wenn diese unter Stickstoff entstehen | ✅ OK: nahe 1.0<br>❌ NOK: verzerrt (z. B. 0.5–0.7) |
| `pad_distance`   | **Abstand zur Padkante** (in mm) | Kritischer Faktor für Schlupf. Wenn die Lotkugel zu nah an einem Lötpad liegt, besteht Kurzschlussgefahr. | ✅ OK: > 0.15 mm<br>❌ NOK: < 0.05 mm |

Diese Merkmale spiegeln typische Prüfkriterien gemäß IPC-A-610 wider.
